IMPORT

In [89]:
import pandas as pd
import csv
import random 


FUNCTIONS

FUNCTION LEGGIDATI

In [90]:

def LeggiDati(filename):

    df=pd.read_csv(filename,header=0)

    corse=[]
    fermate=df.columns[2:].tolist()  #creo una lista con le fermate prendendo dalla seconda colonna in avanti

    for _,riga in df.iterrows():  #iterrows itera per indice e contenuto della riga, con i nomi delle colonne come chiavi.
        autista=riga.iloc[0]
        passeggeri=[]
        for dato in riga[2:]:
            if pd.isna(dato) or dato == '--': # il comando .isna() serve per capire se il valore è nullo
                passeggeri.append((0, 0))
            elif isinstance(dato, str) and '/' in dato: # il comando isintance(dato,str) controlla se dato è una stringa-> ritorna vero o falso
                saliti, scesi = map(int, dato.split('/')) # il comando map(funzione,iterabile) serve per applicare una funzione a ogni elemento di un iterabile
                passeggeri.append((saliti, scesi)) 
            elif isinstance(dato, str) and dato.startswith('-'):
                scesi = int(dato.rstrip('-'))
                passeggeri.append((0, scesi))
            else:
                    try:
                        saliti = int(dato)
                        passeggeri.append((saliti, 0))
                    except:
                        passeggeri.append((0, 0))

        corsa = {'autista': autista, 'passeggeri': passeggeri}
        corse.append(corsa)
    
    return df,corse,fermate




#for corsa in corse:
   # print(corsa)

FUNCTION ESTRAI_SALITI_SCESI

In [91]:
def estrai_saliti_scesi(val):
    if val == '--' or pd.isna(val):
        return (0, 0)
    elif '/' in val:
        saliti, scesi = val.split('/')
        return (int(saliti), -int(scesi))
    elif val.startswith('-'):
        return (0, -int(val.strip('-')))
    else:
        try:
            return (int(val), 0)
        except:
            return (0, 0)

FUNCTION ESTRAI_SALITI_SCESI_CLA

In [92]:
def estrai_saliti_scesi_cla(val):
    if val == '--' or pd.isna(val):
        return ('0/0', 0,0)
    elif '/' in val:
        saliti, scesi = val.split('/')
        return (val,int(saliti), -int(scesi))
    elif val.startswith('-'):
        return (f"0/{int(val.strip('-'))}",0, -int(val.strip('-')))
    else:
        try:
            return (f'{int(val)}/0',int(val), 0)
        except:
            return ('0/0',0, 0)

FUNCTION LEGGINOMI

In [93]:
def LeggiNomi(filename): 
    df=pd.read_csv(filename, sep=':',header=None, names=['Code','Name'])
    dict_nomi={}
    for _,riga in df.iterrows():
        dict_nomi[riga.iloc[0]]=riga.iloc[1]
    #print(dict_nomi)

    return df, dict_nomi

GET DATA

In [94]:
df_db= LeggiNomi('database.txt')[0]
df_db2=LeggiNomi('database.txt')[0]
df_ae=LeggiDati('dati_aeroporto_torino.csv')[0]
# i read.csv sono dentro le funzioni

GET TOTAL NUMBER OF PASSENGERS

In [95]:

df_ae_melt=pd.melt(df_ae,id_vars=['ID', 'Time'],var_name='Fermata_ID',value_name='Passeggeri')  #idvars =identificativo di oggni riga    varname=come chiamo la nuova colonna che prende i nomi delle colonne passate(eccetto quelle identificative) 
df_ae_melt.rename(columns={'ID':'Autista_ID'}, inplace=True)

df_ae_melt['Passeggeri2'] = df_ae_melt['Passeggeri'].astype(str).str.strip() #rimuovo spazi e mi assicuro che siano stringhe, così le converto con la prossima funzione in interi
df_ae_melt[['Passeggeri2','Saliti', 'Scesi']] = df_ae_melt['Passeggeri2'].apply(estrai_saliti_scesi_cla).apply(pd.Series) #applico la funzione estrai_saliti_scesi a tutte le righe

df_ae_melt['Totale_Passeggeri']=df_ae_melt['Saliti'] - (df_ae_melt['Scesi'])
display(df_ae_melt.tail(10))


,Autista_ID,Time,Fermata_ID,Passeggeri,Passeggeri2,Saliti,Scesi,Totale_Passeggeri
638,A139,21:45,F013,-16,0/16,0,-16,16
639,A006,22:00,F013,-12,0/12,0,-12,12
640,A212,22:15,F013,-11,0/11,0,-11,11
641,A021,22:30,F013,-13,0/13,0,-13,13
642,A022,22:45,F013,-4,0/4,0,-4,4
643,A109,23:00,F013,-11,0/11,0,-11,11
644,A200,23:15,F013,-11,0/11,0,-11,11
645,A010,23:30,F013,-11,0/11,0,-11,11
646,A139,23:45,F013,-4,0/4,0,-4,4
647,A006,00:00,F013,-5,0/5,0,-5,5


ASSOCIATED DF_DB AND DF_AE_MELT

In [96]:
df_db.rename(columns={'Code':'Autista_ID'}, inplace=True)
df_db2.rename(columns={'Code':'Fermata_ID'},inplace=True)
#display(df_db.head())

''' PRIMO JOIN'''
df_joined=df_ae_melt.merge(df_db, how='left', on='Autista_ID')
df_joined=df_joined[['Autista_ID','Name','Time','Fermata_ID','Passeggeri','Passeggeri2','Saliti','Scesi','Totale_Passeggeri']] #ho spostato la colonna 'Name' affianco a quella di 'Autista_ID'
df_joined.rename(columns={'Name':'Autista_Name'}, inplace=True)
#display(df_joined.sort_values(by='Totale_Passeggeri'))
display(df_joined)
#display(df_joined.iloc[50:100])


,Autista_ID,Autista_Name,Time,Fermata_ID,Passeggeri,Passeggeri2,Saliti,Scesi,Totale_Passeggeri
0,A012,Eliana Capuana,06:15,F042,13,13/0,13,0,13
1,A003,Giacobbe Comboni,06:30,F042,13,13/0,13,0,13
2,A041,Adele Venditti,06:45,F042,16,16/0,16,0,16
3,A240,Danilo Bazzi,07:00,F042,13,13/0,13,0,13
4,A119,Ida Contarini,07:15,F042,3,3/0,3,0,3
...,...,...,...,...,...,...,...,...,...
643,A109,Felipe Casal Muñoz,23:00,F013,-11,0/11,0,-11,11
644,A200,Elladio Endrizzi,23:15,F013,-11,0/11,0,-11,11
645,A010,Barbara Bush,23:30,F013,-11,0/11,0,-11,11
646,A139,Filippo Gregorio,23:45,F013,-4,0/4,0,-4,4


ASSOCIATED DF_JOINED AND DF_DB2

In [97]:
'''SECONDO JOIN'''
df_joined2=df_joined.merge(df_db2, on='Fermata_ID')
df_joined2.rename(columns={'Name':'Fermata_Name'}, inplace=True)
df_joined2=df_joined2[['Autista_ID','Autista_Name','Time','Fermata_ID','Fermata_Name','Passeggeri','Passeggeri2','Saliti','Scesi','Totale_Passeggeri']]
display(df_joined2)

,Autista_ID,Autista_Name,Time,Fermata_ID,Fermata_Name,Passeggeri,Passeggeri2,Saliti,Scesi,Totale_Passeggeri
0,A012,Eliana Capuana,06:15,F042,Aeroporto,13,13/0,13,0,13
1,A003,Giacobbe Comboni,06:30,F042,Aeroporto,13,13/0,13,0,13
2,A041,Adele Venditti,06:45,F042,Aeroporto,16,16/0,16,0,16
3,A240,Danilo Bazzi,07:00,F042,Aeroporto,13,13/0,13,0,13
4,A119,Ida Contarini,07:15,F042,Aeroporto,3,3/0,3,0,3
...,...,...,...,...,...,...,...,...,...,...
643,A109,Felipe Casal Muñoz,23:00,F013,Torino P. Nuova,-11,0/11,0,-11,11
644,A200,Elladio Endrizzi,23:15,F013,Torino P. Nuova,-11,0/11,0,-11,11
645,A010,Barbara Bush,23:30,F013,Torino P. Nuova,-11,0/11,0,-11,11
646,A139,Filippo Gregorio,23:45,F013,Torino P. Nuova,-4,0/4,0,-4,4


GET MAX/MIN PASSENGERS

In [98]:
'''RACCOLGO SALITI E SCESI'''
SalitiRaccolti=df_joined2.groupby('Fermata_Name')['Saliti'].sum()
#display(SalitiRaccolti)
ScesiRaccolti=df_joined2.groupby('Fermata_Name')['Scesi'].sum()
#display(ScesiRaccolti)

max_saliti=SalitiRaccolti.max()
#display(max_saliti)
max_scesi=-(ScesiRaccolti.min())

fermata_max= SalitiRaccolti.idxmax()
#display(indice_fermata_max)
fermata_min=ScesiRaccolti.idxmin()

GET SOLUTION

In [99]:


'''PRINT E SOLUZIONE'''
#MaxSaliti=df_fermateRaccolte['Saliti'].max()
#display(MaxSaliti)
print(f'La fermata con più saliti è {fermata_max} con {max_saliti} passeggeri saliti ')
print(f'La fermata con più scesi è {fermata_min} con {max_scesi} passeggeri scesi')

#print(df_joined.groupby('Autista_ID')['Saliti'].sum())
SalitiXAutista=df_joined.groupby('Autista_ID')['Saliti'].sum().to_dict() #creo dizionario con chiave autista id e valore la somma dei saliti per quell'autista id
#print(SalitiXAutista)

SalitiXAutistaOrdinato= dict(sorted(SalitiXAutista.items(), key=lambda item: item[1], reverse=True)) #ordino il dizionario in modo decrescente
print(SalitiXAutistaOrdinato)
print('Lista di autisti ordinata in base al numero di passeggeri saliti in una giornata:')

for Autista_ID in SalitiXAutistaOrdinato:
    print(f'{Autista_ID} con {SalitiXAutistaOrdinato[Autista_ID]} passeggeri')




La fermata con più saliti è  Aeroporto con 1626 passeggeri saliti 
La fermata con più scesi è  Torino P. Nuova con 1755 passeggeri scesi
{'A003': 335, 'A240': 316, 'A119': 301, 'A012': 296, 'A041': 289, 'A015': 284, 'A008': 257, 'A154': 246, 'A021': 163, 'A200': 149, 'A010': 140, 'A022': 117, 'A212': 111, 'A139': 107, 'A109': 98, 'A006': 87}
Lista di autisti ordinata in base al numero di passeggeri saliti in una giornata:
A003 con 335 passeggeri
A240 con 316 passeggeri
A119 con 301 passeggeri
A012 con 296 passeggeri
A041 con 289 passeggeri
A015 con 284 passeggeri
A008 con 257 passeggeri
A154 con 246 passeggeri
A021 con 163 passeggeri
A200 con 149 passeggeri
A010 con 140 passeggeri
A022 con 117 passeggeri
A212 con 111 passeggeri
A139 con 107 passeggeri
A109 con 98 passeggeri
A006 con 87 passeggeri
